# 📊 Productivity Analysis with AI
This notebook reads task data from `tasks_export.csv`, generates a productivity chart, and provides insights using an AI model (e.g., `llama3.1:8b` via Ollama).

In [ ]:
# Step 1: Load task data
import pandas as pd
from datetime import datetime

df = pd.read_csv("tasks_export.csv")
df["Created At"] = pd.to_datetime(df["Created At"])
df["Completed At"] = pd.to_datetime(df["Completed At"], errors="coerce")
df.head()

In [ ]:
# Step 2: Compute daily statistics
from collections import defaultdict
import pandas as pd

created_per_day = df["Created At"].dt.date.value_counts().to_dict()
completed_per_day = df["Completed At"].dropna().dt.date.value_counts().to_dict()

all_dates = pd.date_range(
    df["Created At"].min().date(),
    max(df["Created At"].max(), df["Completed At"].max()).date()
)

created_counts = [created_per_day.get(date.date(), 0) for date in all_dates]
completed_counts = [completed_per_day.get(date.date(), 0) for date in all_dates]

# Calculate cumulative pending tasks
pending_counts = []
current_pending = 0
for c, d in zip(created_counts, completed_counts):
    current_pending += c - d
    pending_counts.append(max(0, current_pending))

In [ ]:
# Step 3: Plot productivity trends
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

plt.figure(figsize=(12, 6))
plt.plot(all_dates, created_counts, label="Created", marker="o")
plt.plot(all_dates, completed_counts, label="Completed", marker="o")
plt.plot(all_dates, pending_counts, label="Pending", linestyle="--", color="red")
plt.xlabel("Date")
plt.ylabel("Tasks")
plt.title("Productivity Trends")
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))
plt.tight_layout()
plt.show()

In [ ]:
# Step 4: Generate AI Insight
import ollama

total_created = sum(created_counts)
total_completed = sum(completed_counts)
total_pending = total_created - total_completed

prompt = f"""
Productivity Summary:
- Tasks created: {total_created}
- Tasks completed: {total_completed}
- Tasks pending: {total_pending}

Write a short, motivational productivity analysis.
"""

response = ollama.chat(model="llama3.1:8b", messages=[{"role": "user", "content": prompt}])
print("✨ AI Insight:\n", response["message"]["content"])